In [ ]:
import re
import cv2
import csv
import time
import pyARIS
import threading
import subprocess
import numpy as np
import datumaro as dm
from my_utils import *
from PIL import Image
from tqdm import tqdm
from dateutil import parser
import matplotlib.pyplot as plt
from multiprocessing import Process

sonarDataDir = "./sonar/"
outputDir = "./output/"

def preprocess_data_and_generate_csv_file():
    output_video_folder_path = os.path.join(outputDir, "video")
    create_folder(output_video_folder_path)
    r = "(.+)_(.+)\.zip"

    with open(os.path.join(outputDir, "fileInfo.csv"), "w") as f:
        csvWriter = csv.writer(f)
        csvWriter.writerow(["filename", "vid_path", "anno_path"])

        for item in os.listdir(os.path.join(sonarDataDir, "annotation")):
            match = re.match(r, item)
            date = parser.parse(match[1])
            time = match[2]

            ARISfolderName = "ARIS_%s" %(date.strftime("%Y_%m_%d"))
            ARISFolderPath = os.path.join(sonarDataDir, "ARIS", ARISfolderName)
            ARISFilePath = os.path.join(ARISFolderPath, "%s_%s.aris" %(date.strftime("%Y-%m-%d"), time))
            
            # From annotation folder, find the corresponding ARIS file and preprocess
            if os.path.exists(ARISFolderPath) and os.path.exists(ARISFilePath):
                print("Processing %s" %(ARISFilePath))
                ARISFileName = os.path.splitext(os.path.basename(ARISFilePath))[0]
                videoFolderPath = os.path.join(output_video_folder_path, ARISfolderName)
                create_folder(videoFolderPath)

                ARISdata, _ = pyARIS.DataImport(ARISFilePath)
                pyARIS.VideoExportOriginal(ARISdata, filename=os.path.join(videoFolderPath, "%s.mp4" %(ARISFileName)))

                csvWriter.writerow([ARISFileName, os.path.join(videoFolderPath, "%s.mp4" %(ARISFileName)), os.path.join(sonarDataDir, "annotation", item)])

def convert_annotation_to_coco():
    pass

if __name__ == "__main__":
    preprocess_data_and_generate_csv_file()

    
    # # create mp4 file from aris file
    # filePath = "./sonar/ARIS/ARIS_2020_05_31/2020-05-31_015000.aris"
    # ARISdata, _ = pyARIS.DataImport(filePath)
    # pyARIS.VideoExportOriginal(ARISdata, filename="video.mp4")

    # # convert from datumaro to coco
    # dataset = dm.Dataset.import_from('./datum_proj/', 'datumaro')
    # dataset.export('save_dir', 'coco')